In [ ]:
#| default_exp climate

Weather: Lets give them something to talk about

#| export


## Climate Modeling for Hex Terrain

### Overview

We built a climate classification system that computes temperature, precipitation, and biome type for each hex in a terrain map. The system uses:
- **Latitude** and **elevation** for temperature
- **Wind-based moisture flow** for precipitation
- **Temperature + precipitation thresholds** for climate classification

### Temperature Model

**Factors:**
1. **Latitude** - Temperature decreases toward poles using cosine scaling
2. **Elevation** - Standard lapse rate of ~6.5°C per 1000m
3. **Coastal moderation** - Ocean moderates temperature swings (smaller daily range near coast)

**Key function:** `compute_temperature(lat_range, base_temp_low, base_temp_high, temp_lapse_rate)`

### Precipitation Model

**Algorithm:** Wind tree with moisture flow

1. **Build wind tree** - BFS from coastal (ocean) hexes in the wind direction
2. **Flow moisture** - Each hex receives moisture from its upwind parent
3. **Orographic effects:**
   - Climbing = more rain (orographic lift)
   - Descending = rain shadow (less rain)
   - Flat = normal decay

**Key insight:** We run from multiple wind angles (270°, 280°, 260°) and weight-average the results to avoid artifacts.

**Tuning parameters:**
- `initial_moisture=3000` - Starting moisture at coast (mm)
- `travel_decay=0.92` - Moisture retained per hex traveled
- `orographic_factor=0.4` - Fraction dropped when climbing

**Key functions:**
- `build_wind_tree(wind_angle)` - Creates parent/child relationships
- `compute_precipitation_flow(...)` - Flows moisture through tree
- `compute_and_visualize_rainfall(...)` - Multi-angle wrapper

### Climate Classification

**Enum:**
```python
class Climate(Enum):
    MARINE = 0      # Ocean/sea
    FRESHWATER = 1  # Lakes/rivers (not yet implemented)
    TUNDRA = 2      # Cold (tempHigh < 10°C)
    DESERT = 3      # Dry (rain < 150mm)
    GRASSLAND = 4   # Moderate (150-600mm rain)
    FOREST = 5      # Wet temperate (> 600mm rain)
    JUNGLE = 6      # Tropical (> 1500mm or hot+wet)
```

**Decision order matters:** Marine → Tundra → Desert → Jungle → Forest → Grassland

### Debugging Tools

- `summarize_climate()` - Prints distribution and stats
- `debug_precipitation()` - Shows wind tree and flow stats

### Known Issues / Future Work

1. **Freshwater** - Need river tracing to identify lakes/rivers
2. **Pre-loaded data** - Files may have old rainfall; use `force_recompute=True`
3. **California specifics** - Thresholds tuned for Mediterranean climate; other regions may need adjustment
4. **Seasonality** - Current model is annual average; could add wet/dry seasons

### Quick Start

```python
demo = TerraDemo()
terrain = demo.california()
terrain.compute_and_visualize_rainfall(initial_moisture=3000, travel_decay=0.92)
terrain.compute_temperature(lat_range=(32.5, 42.0))
terrain.compute_climate()
terrain.summarize_climate()
terrain.add_climate_overlay()
terrain.hexGrid.builder.show()
```

---


In [ ]:
#| export
import numpy as np
import sys
import os
import math
import random

#data
from collections import namedtuple
from dataclasses import dataclass,  field, asdict
from typing import List
from enum import Enum

#Jeremy
from dialoghelper import * 
from fastcore.basics import patch
from fasthtml.common import *
from fasthtml.jupyter import *
import httpx

#custom
import inspect
import copy
import colorsys

from HexMagic.styles import StyleCSS, SVGBuilder, SVGLayer, SVGPatternLoader, preview, app, StyleDemo, LayerAnimation
from HexMagic.primitives import MapCord, MapSize, MapRect, MapPath, Hex, HexGrid, HexWrapper, HexPosition, hexBackground, HexRegion, windy_edge
from HexMagic.terrain import  TerraDemo, Terrain, GeoBounds, ClimatePreset
#from terrainpattern import TerrainPatterns
from HexMagic.river import River, SoilSystem, ErosionModel

In [ ]:
#| export
from HexMagic.terrainpatterns import TerrainPatterns

In [ ]:
#| export
from HexMagic.river import River, RiverDemo

In [ ]:
#| export
import math
from collections import deque

In [ ]:
#| export
import orographic_precipitation as op



So we are going to reimplement a great deal of what we had using the model based upon this link `https://journals.ametsoc.org/view/journals/atsc/61/12/1520-0469_2004_061_1377_altoop_2.0.co_2.xml` the goal is to find compontents that we can compute climates/biomes 

class Climate(Enum):
    MARINE = 0        # Ocean/sea
    FRESHWATER = 1    # Lakes/rivers
    TUNDRA = 2        # Cold, low precipitation
    DESERT = 3        # Hot/cold, very low precipitation
    GRASSLAND = 4     # Moderate temp, moderate precipitation
    FOREST = 5        # Moderate temp, high precipitation
    JUNGLE = 6        # Hot, very high precipitation
I know that we can use things like elevation, temperature, preciptation to start doing this. I don't know if there are other factors that would be helpful. one thing that is good to understand is our hexgrid system which has rings of hexes around any hex which makes it easy to compute things like shorest path or sampling. I am going to include some sample code here which shows how we average fields across our data.

`
@patch
def downsample_field(self: Terrain, field_array, sample_radius=2, method='weighted_avg'):
    """Downsample any field array using hex ring sampling.
    
    Args:
        field_array: numpy array of values to downsample (same length as hexes)
        sample_radius: How many rings to sample
        method: 'weighted_avg', 'max', 'min', or 'mode' (for categorical)
    
    Returns:
        Downsampled array matching new grid size
    """
    old_grid = self.hexGrid
    factor = sample_radius + 1
    new_rows = old_grid.nRows // factor
    new_cols = old_grid.nCols // factor
    
    if new_rows < 2 or new_cols < 2:
        print("Warning: sample_radius too large")
        return field_array
    
    # Calculate new grid size
    new_size = new_rows * new_cols
    new_field = np.zeros(new_size)
    
    for new_idx in range(new_size):
        new_row = new_idx // new_cols
        new_col = new_idx % new_cols
        
        old_row = new_row * factor
        old_col = new_col * factor
        
        if old_row >= old_grid.nRows or old_col >= old_grid.nCols:
            continue
            
        center_idx = old_row * old_grid.nCols + old_col
        
        # Collect values from rings
        values = []
        weights = []
        
        for ring_num in range(sample_radius + 1):
            weight = 1.0 / (ring_num + 1)
            
            if ring_num == 0:
                ring_hexes = [center_idx]
            else:
                ring_positions = HexPosition(0, 0, 0).ring(ring_num)
                ring_hexes = [old_grid.hexposition_to_index(hp, center_idx) 
                             for hp in ring_positions]
                ring_hexes = [h for h in ring_hexes if h >= 0]
            
            for hex_idx in ring_hexes:
                values.append(field_array[hex_idx])
                weights.append(weight)
        
        if not values:
            continue
            
        values = np.array(values)
        weights = np.array(weights)
        
        # Apply aggregation method
        if method == 'weighted_avg':
            new_field[new_idx] = np.average(values, weights=weights)
        elif method == 'max':
            new_field[new_idx] = np.max(values)
        elif method == 'min':
            new_field[new_idx] = np.min(values)
        elif method == 'mode':
            # For categorical data - weighted mode
            unique_vals = np.unique(values)
            best_val = unique_vals[0]
            best_weight = 0
            for val in unique_vals:
                val_weight = np.sum(weights[values == val])
                if val_weight > best_weight:
                    best_weight = val_weight
                    best_val = val
            new_field[new_idx] = best_val
    
    return new_field
`
@patch
def downsample_ring(self: Terrain, sample_radius=2, method='weighted_avg'):
    """Downsample terrain using hex ring sampling.
    
    Args:
        sample_radius: How many rings to sample (higher = more smoothing)
        method: 'weighted_avg', 'max', 'min', or 'mode' (for categorical)
    
    Returns:
        New Terrain with fewer hexes covering same area
    """
    old_grid = self.hexGrid
    
    # Calculate new grid dimensions
    factor = sample_radius + 1
    new_rows = old_grid.nRows // factor
    new_cols = old_grid.nCols // factor
    
    if new_rows < 2 or new_cols < 2:
        print("Warning: sample_radius too large for this terrain")
        return self
    
    # Keep same bounds, just use larger hex radius
    new_radius = old_grid.radius * factor
    
    # Create new terrain with same bounds but larger hexes
    new_terrain = Terrain(old_grid.bounds, radius=new_radius)
    
    # Copy style properties
    new_terrain.colorLevels = self.colorLevels.copy() if self.colorLevels else None
    new_terrain.elevationDelta = self.elevationDelta
    new_terrain.seaLevel = self.seaLevel
    
    # Use the downsample_field method to sample elevations
    new_terrain.elevations = self.downsample_field(
        self.elevations, 
        sample_radius=sample_radius, 
        method=method
    )
    
    # Add color styles
    if new_terrain.colorLevels:
        for color in new_terrain.colorLevels:
            new_terrain.hexGrid.builder.add_style(color)
    
    return new_terrain
    `
    So I think we need things that are going to create fields of data that we can then use to compute things like temperature, precipitation. we can take advantage of some of the existing mapping code to see progress. for intance this shows climate

    @patch
def add_climate_overlay(self: Terrain, layer_name="climate"):
    """Visualize climate zones with appropriate colors."""
    
    # Define colors for each climate type
    climate_colors = {
        Climate.MARINE.value: "#1e88e5",      # Blue - ocean
        Climate.FRESHWATER.value: "#42a5f5",  # Light blue - lakes
        Climate.TUNDRA.value: "#e3f2fd",      # Pale blue - tundra
        Climate.DESERT.value: "#fdd835",      # Yellow - desert
        Climate.GRASSLAND.value: "#9ccc65",   # Light green - grassland
        Climate.FOREST.value: "#2e7d32",      # Dark green - forest
        Climate.JUNGLE.value: "#1b5e20",      # Very dark green - jungle
    }
    
    if 'climate' not in self.fields:
        self.compute_climate()
    
    # Create patterns for each climate type
    patternGen = TerrainPatterns(self)
    colors = [climate_colors[i] for i in range(len(Climate))]
    patterns = patternGen.ballDensity(len(colors), fills=colors, prefix="climate")
    
    # Generate overlay
    climate_indices = self.fields['climate'].astype(int)
    overlay = self.makeOverlay(climate_indices, patterns)
    
    # Add to builder
    self.builder.adjust(layer_name, overlay)
    `

    Let's think about how we would go through things.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠



<details class='tool-usage-details'>

```json
{
  "id": "toolu_016pwTWX4YWFHRMBxHHm6k94",
  "call": {
    "function": "read_url",
    "arguments": {
      "url": "https://journals.ametsoc.org/view/journals/atsc/61/12/1520-0469_2004_061_1377_altoop_2.0.co_2.xml"
    }
  },
  "result": "\"# JavaScript is disabled\\n\\nIn order to continue, we need to verify that you're not a robot. This requires JavaScript. Enable JavaScript and then reload the page. \\n\""
}
```

</details>

Let me search for information about this paper and the Linear Theory of Orographic Precipitation model:No worries - I'm familiar with the Smith & Barstad (2004) model! And I see you already have the `orographic_precipitation` package installed. Let me help you think through this systematically.

**The Linear Theory of Orographic Precipitation** is a physics-based model that computes precipitation from:

1. **Terrain elevation** (the DEM)
2. **Wind speed and direction** 
3. **Atmospheric parameters** (moisture, stability, scale heights)
4. **Grid resolution** (dx, dy)

This is actually more principled than your current wind-tree approach because it uses Fourier transforms to solve the physics equations.

**For computing Climate/Biomes, you need these primary fields:**

| Field | Source | Notes |
|-------|--------|-------|
| **Elevation** | Already have | Your terrain |
| **Temperature** | Compute from lat + elevation | Lapse rate ~6.5°C/1000m |
| **Precipitation** | Smith-Barstad model | Uses `orographic_precipitation` |
| **Distance to coast** | BFS from ocean hexes | For continentality |
| **Latitude** | From bounds | Temperature gradient |

**Additional factors that could help:**

- **Seasonality** - Wet/dry seasons (could run model for summer vs winter wind patterns)
- **Continentality** - Distance from ocean affects temp range and moisture
- **Aspect/slope** - South-facing vs north-facing affects local temp

**Let's start by getting the Smith-Barstad model working with your hex grid.**

The key challenge: `compute_orographic_precip` expects a **2D numpy array** (regular grid), but your terrain uses a hex grid. You'll need to:

1. Convert hex elevations → 2D array
2. Run the orographic model
3. Sample results back to hex positions

Here's a sketch of the approach:

```python
@patch
def elevation_to_grid(self: Terrain):
    """Convert hex elevations to 2D numpy array for orographic model."""
    grid = self.hexGrid
    # Create 2D array matching hex grid dimensions
    elev_2d = self.elevations.reshape(grid.nRows, grid.nCols)
    return elev_2d

@patch  
def compute_orographic_rainfall(self: Terrain, 
                                 wind_speed=10.0,      # m/s
                                 wind_dir=270.0,       # degrees (270=westerly)
                                 latitude=37.0,        # for Coriolis
                                 precip_base=0.5,      # background precip mm/h
                                 annual_hours=8760):   # convert to annual mm
    """Compute precipitation using Smith-Barstad linear model."""
    import orographic_precipitation as op
    
    # Get 2D elevation array
    elev_2d = self.elevation_to_grid()
    
    # Grid spacing in meters (approximate from hex radius)
    dx = dy = self.hexGrid.radius * 1.5  # rough conversion
    
    # Run the model
    precip_rate = op.compute_orographic_precip(
        elev_2d, dx, dy,
        latitude=latitude,
        wind_speed=wind_speed,
        wind_dir=wind_dir,
        precip_base=precip_base,
        precip_min=0.01
    )
    
    # Convert mm/h to annual mm
    annual_precip = precip_rate * annual_hours
    
    # Flatten back to hex array
    self.fields['precipitation'] = annual_precip.flatten()
    
    return self.fields['precipitation']
```

**Questions before we dive in:**

1. What's your hex grid's approximate real-world resolution? (meters per hex)
2. Do you want to start with the basic model, or also include multiple wind directions right away?
3. Should we also refactor `compute_temperature` to be cleaner, or focus on precipitation first?

## some terrain helpers

In [ ]:
#| export
@patch
def _compute_hex_coordinates(self: Terrain):
    """Compute lat/lon for each hex center."""
    if self.geo is None:
        return
    
    grid = self.hexGrid
    
    # Store lat/lon for each hex
    self.fields['latitude'] = np.zeros(len(self.elevations))
    self.fields['longitude'] = np.zeros(len(self.elevations))
    
    for idx in range(len(self.elevations)):
        row = idx // grid.nCols
        col = idx % grid.nCols
        
        # Normalize to [0, 1]
        row_norm = row / (grid.nRows - 1) if grid.nRows > 1 else 0.5
        col_norm = col / (grid.nCols - 1) if grid.nCols > 1 else 0.5
        
        # Map to geographic coordinates
        # Note: row 0 = top = max latitude
        self.fields['latitude'][idx] = self.geo.lat_max - row_norm * (self.geo.lat_max - self.geo.lat_min)
        self.fields['longitude'][idx] = self.geo.lon_min + col_norm * (self.geo.lon_max - self.geo.lon_min)

@patch
def grid_resolution_meters(self: Terrain):
    """Calculate dx, dy in meters for the grid."""
    if self.geo is None:
        return None, None
    
    # Meters per degree at center latitude
    lat_center = (self.geo.lat_min + self.geo.lat_max) / 2
    lat_rad = math.radians(lat_center)
    
    meters_per_lat = 111132.92  # ~constant
    meters_per_lon = 111132.92 * math.cos(lat_rad)
    
    # Resolution per grid cell
    lat_span = self.geo.lat_max - self.geo.lat_min
    lon_span = self.geo.lon_max - self.geo.lon_min
    
    dy = (lat_span / self.hexGrid.nRows) * meters_per_lat
    dx = (lon_span / self.hexGrid.nCols) * meters_per_lon
    
    return dx, dy



In [ ]:
#| export
@patch
def california_with_geo(self: TerraDemo):
    """Load California terrain with proper geographic bounds."""
    with open("data/templates/ca_nv_elevation_grid.txt", "r") as f:
        terrain = Terrain.decode(f.read())
    
    # Set the geographic bounds that match your data
    terrain.geo = GeoBounds(
        lat_min=32.5,   # Southern California/Nevada border
        lat_max=42.0,   # Northern California/Oregon border
        lon_min=-124.5, # Pacific coast
        lon_max=-114.5  # Eastern Nevada
    )
    
    # Compute hex coordinates
    terrain._compute_hex_coordinates()
    
    return terrain


In [ ]:
#| export
@patch
def compute_precipitation_sb(self: Terrain,
                              wind_speed=15.0,
                              wind_dir=270.0,
                              precip_base=0.3,
                              # Atmospheric parameters with typical values
                              nm=0.005,         # moist stability frequency [1/s]
                              hw=2500.0,        # water vapor scale height [m]
                              cw=0.005,         # uplift sensitivity [kg/m³]
                              conv_time=1000.0, # conversion time [s]
                              fall_time=1000.0 , # fallout time [s]
                              debug = False
                            
                             ):
    """
    Compute orographic precipitation using Smith-Barstad (2004) model.
    
    Args:
        wind_speed: Wind speed in m/s (typical: 10-20)
        wind_dir: Wind direction in degrees (270 = from west)
        precip_base: Background precipitation rate in mm/h (typical: 0.1-1.0)
        nm: Moist stability frequency [1/s] (typical: 0.005)
        hw: Water vapor scale height [m] (typical: 2000-3000)
        cw: Uplift sensitivity [kg/m³] (typical: 0.003-0.006)
        conv_time: Cloud water conversion time [s] (typical: 500-1500)
        fall_time: Hydrometeor fallout time [s] (typical: 500-1500)
    
    Returns:
        Annual precipitation in mm
    """
    
    
    if self.geo is None:
        raise ValueError("Geographic bounds not set. Use terrain.geo = GeoBounds(...)")
    
    # Get 2D elevation array
    grid = self.hexGrid
    elev_2d = self.elevations.reshape(grid.nRows, grid.nCols)
    
    # Get grid resolution in meters
    dx, dy = self.grid_resolution_meters()
    
    if debug:
        print(f"Grid: {grid.nRows} x {grid.nCols}")
        print(f"Resolution: dx={dx:.0f}m (~{dx/1000:.1f}km), dy={dy:.0f}m (~{dy/1000:.1f}km)")
        print(f"Latitude: {self.geo.lat_min}° to {self.geo.lat_max}° (center: {(self.geo.lat_min + self.geo.lat_max)/2:.1f}°)")
        print(f"Wind: {wind_speed} m/s from {wind_dir}°")
        print(f"Background precip: {precip_base} mm/h")
    
    # Run Smith-Barstad model with all required parameters
    precip_rate = op.compute_orographic_precip(
        elev_2d, dx, dy,
        latitude=(self.geo.lat_min + self.geo.lat_max) / 2,
        wind_speed=wind_speed,
        wind_dir=wind_dir,
        precip_base=precip_base,
        precip_min=0.01,
        nm=nm,
        hw=hw,
        cw=cw,
        conv_time=conv_time,
        fall_time=fall_time
    )
    
    # Convert mm/h to annual mm
    hours_per_year = 8760
    annual_precip = precip_rate * hours_per_year
    
    # Flatten and store
    self.fields['precipitation'] = annual_precip.flatten()
    self.fields['precip_rate_mmh'] = precip_rate.flatten()
    
    if debug:
        print(f"\nPrecipitation range: {annual_precip.min():.0f} - {annual_precip.max():.0f} mm/year")
        print(f"Mean: {annual_precip.mean():.0f} mm/year")
    
    return self.fields['precipitation']


In [ ]:
#| export

@patch
def visualize_precipitation(self: Terrain, layer_name="precipitation"):
    """Visualize precipitation with color gradient."""
    if 'precipitation' not in self.fields:
        print("Computing precipitation first...")
        self.compute_precipitation_sb()
    
    precip = self.fields['precipitation']
    
    # Precipitation ranges for California (mm/year)
    thresholds = [0, 150, 300, 500, 750, 1000, 1500, 2000]
    colors = [
        "#8B4513",  # Brown - desert (<150mm)
        "#DEB887",  # Tan - arid (150-300)
        "#F0E68C",  # Khaki - semi-arid (300-500)
        "#9ACD32",  # Yellow-green - dry (500-750)
        "#32CD32",  # Lime green - moderate (750-1000)
        "#228B22",  # Forest green - wet (1000-1500)
        "#006400",  # Dark green - very wet (1500-2000)
        "#004d00",  # Very dark green - rainforest (>2000)
    ]
    
    # Bin precipitation
    precip_indices = np.digitize(precip, thresholds) - 1
    precip_indices = np.clip(precip_indices, 0, len(colors) - 1)
    
    # Create patterns
    patternGen = TerrainPatterns(self)
    patterns = patternGen.ballDensity(len(colors), fills=colors, prefix="precip")
    

    overlay = self.makeOverlay(precip_indices, patterns)
    self.builder.adjust(layer_name, overlay)
    
    return self

In [ ]:
??Terrain.makeOverlay


```python
@patch
def makeOverlay(self:Terrain,data,patterns:[SVGBuilder])->str:
    testBody = ""
    grid = self.hexGrid

    addSet = set()

    for i, patIndex in enumerate(data):
        if patIndex >= 0 and patIndex < len(patterns):
            patName = patterns[patIndex].attributes['id']
            addSet.add(patIndex)

            fill = f"url(#{patName})"

            ret = "<polygon points=\""
            hex = grid.hexes[i]
            for point in hex.vertices():
                ret += f"{point.x:.0f},{point.y:.0f} "
            ret += f"\" style=\"fill:{fill}\""
            ret += "/>"

            testBody += "\t" +  ret + "\n"

    aList = list(addSet)
    aList.sort()
    for patIndex in aList:
        grid.builder.add_definition(patterns[patIndex])

    return testBody
```

**File:** `~/HexMagic/HexMagic/terrainpatterns.py`

### Test method

In [ ]:
@patch
def rainDemo(self:TerraDemo,debug=True):
    # Load California with geographic bounds
    demo = TerraDemo()
    terrain = demo.california_with_geo()

    # Check that coordinates were computed
    if debug:
        print(f"Latitude range: {terrain.fields['latitude'].min():.2f}° to {terrain.fields['latitude'].max():.2f}°")
        print(f"Longitude range: {terrain.fields['longitude'].min():.2f}° to {terrain.fields['longitude'].max():.2f}°")

    terrain.compute_precipitation_sb(
        wind_speed=10.0,      
        wind_dir=270.0,       
        precip_base=0.005,    # 5x lower! (44 mm/year background)
        nm=0.015,             # Even more stable
        hw=1500.0,            # Lower moisture scale
        cw=0.0005,            # 4x less uplift sensitivity
        conv_time=3000.0,     
        fall_time=3000.0 ,
        debug = debug     
    )

    terrain.visualize_precipitation()
    return terrain.hexGrid.builder.show()


In [ ]:
TerraDemo().rainDemo(False)

In [ ]:
#| export


@patch
def climate_readiness(self: Terrain):
    """Check what fields are available for climate classification."""
    required = ['elevation', 'temperature', 'precipitation']
    optional = ['latitude', 'longitude', 'distance_to_coast']
    
    print("=== CLIMATE READINESS ===")
    print("\nRequired fields:")
    print(f"  elevation:     {'✓' if len(self.elevations) > 0 else '✗'}")
    print(f"  temperature:   {'✓' if 'temperature' in self.fields else '✗'}")
    print(f"  precipitation: {'✓' if 'precipitation' in self.fields else '✗'}")
    
    print("\nOptional fields:")
    for field in optional:
        print(f"  {field:20s} {'✓' if field in self.fields else '✗'}")
    
    print("\nAvailable fields:")
    for field in sorted(self.fields.keys()):
        print(f"  - {field}")


@patch
def climate_distribution(self: Terrain):
    """Show distribution of climate zones (after classification)."""
    if 'climate' not in self.fields:
        return "Climate not computed yet"
    
    from collections import Counter
    climate_data = self.fields['climate'].astype(int)
    counts = Counter(climate_data)
    total = len(climate_data)
    
    print("\n=== CLIMATE DISTRIBUTION ===")
    climate_names = ['MARINE', 'FRESHWATER', 'TUNDRA', 'DESERT', 
                     'GRASSLAND', 'FOREST', 'JUNGLE']
    
    for i, name in enumerate(climate_names):
        count = counts.get(i, 0)
        pct = 100 * count / total
        bar = '█' * int(40 * count / max(counts.values()))
        print(f"{name:12s}: {bar} {count:4d} ({pct:5.1f}%)")


In [ ]:
#| export
@patch
def mauiMap(self:TerraDemo,debug = False):
    """Load Maui terrain with proper geographic bounds."""
    with open("data/templates/maui.txt", "r") as f:
        terrain = Terrain.decode(f.read())
    
    # Set Maui's geographic bounds
    terrain.geo = GeoBounds(
        lat_min=20.57,   # Southern tip (near Makena)
        lat_max=21.03,   # Northern tip (near Kahakuloa)
        lon_min=-156.69, # Western tip (West Maui)
        lon_max=-155.97  # Eastern tip (Haleakalā/Hāna)
    )
    
    # Compute hex coordinates
    terrain._compute_hex_coordinates()
    
    # Maui-specific precipitation model
    # Trade winds from northeast at ~50-60 degrees
    terrain.compute_precipitation_sb(
    wind_speed=8.0,        # Slightly stronger trade winds
    wind_dir=50.0,         
    precip_base=0.15,      # MUCH higher base moisture (tropical ocean)
    nm=0.008,              # Less stable (more convection)
    hw=2500.0,             # Higher moisture scale height
    cw=0.003,              # MUCH stronger orographic effect
    conv_time=1000.0,      # Faster conversion
    fall_time=1000.0,       # Faster fallout
    debug = debug
)

    return terrain

@patch
def maui(self:TerraDemo,debug=False):
    terrain = self.mauiMap(debug=debug)
    if debug:
        terrain.field_summary('precipitation')
    return len(terrain.hexGrid.builder.xml())


In [ ]:
!ls data/templates

BayArea.txt		  grand_canyon.txt  normandy.txt
agincourt.txt		  iwo_jima.txt	    pompeii.txt
ca_nv_elevation_grid.txt  maui.txt	    san_francisco.txt


In [ ]:
TerraDemo().maui(True)

Grid: 100 x 70
Resolution: dx=1069m (~1.1km), dy=511m (~0.5km)
Latitude: 20.57° to 21.03° (center: 20.8°)
Wind: 8.0 m/s from 50.0°
Background precip: 0.15 mm/h

Precipitation range: 4 - 15267 mm/year
Mean: 2466 mm/year

=== PRECIPITATION ===
Range:  3.8 to 15266.5
Mean:   2466.4
Median: 1801.9
StdDev: 3101.7
Percentiles: 10%=87.6, 25%=87.6, 75%=3019.4, 90%=6558.5

Distribution:
      3.8 -  1911.6: ████████████████████████████████████████ (3741)
   1911.6 -  3819.5: ████████████████████ (1908)
   3819.5 -  5727.3: █████ (512)
   5727.3 -  7635.1: ███ (291)
   7635.1 -  9543.0: █ (164)
   9543.0 - 11450.8: █ (140)
  11450.8 - 13358.7: █ (132)
  13358.7 - 15266.5: █ (112)


1440

## Temperature

In [ ]:
#| export
@patch
def compute_distance_from_coast(self: Terrain):
    """Calculate distance from coast for each hex using BFS."""
    from collections import deque
    
    n_hexes = len(self.elevations)
    distances = np.full(n_hexes, np.inf)
    
    # Find all coastal hexes (at or below sea level)
    coastal = [i for i in range(n_hexes) if self.elevations[i] <= 0]
    
    # BFS to calculate distances
    queue = deque()
    for coast_idx in coastal:
        distances[coast_idx] = 0
        queue.append((coast_idx, 0))
    
    visited = set(coastal)
    
    while queue:
        current_idx, dist = queue.popleft()
        
        # Check all neighbors
        neighbors = self.ring(current_idx, 1)
        for neighbor in neighbors:
            if neighbor >= 0 and neighbor not in visited:
                distances[neighbor] = dist + 1
                visited.add(neighbor)
                queue.append((neighbor, dist + 1))
    
    self.fields['distance_to_coast'] = distances.astype(float)
    return self.fields['distance_to_coast']



In [ ]:
#| export
@patch
def compute_temperature(self: Terrain,
                        base_temp_south=20.0,    # Southern coast 
                        base_temp_north=14.0,    # Northern coast  
                        lapse_rate=6.5):         # °C per 1000m
    """
    Temperature model for California - mean annual temperature.
    Continentality affects seasonal range, not annual mean.
    """
    if 'distance_to_coast' not in self.fields:
        self.compute_distance_to_coast()
    
    # Latitude-based baseline
    if 'latitude' in self.fields and self.geo:
        lat_fraction = (self.fields['latitude'] - self.geo.lat_min) / (self.geo.lat_max - self.geo.lat_min)
        base_temp = base_temp_south + (base_temp_north - base_temp_south) * lat_fraction
    else:
        base_temp = np.full(len(self.elevations), (base_temp_south + base_temp_north) / 2)
    
    # Apply elevation lapse rate
    temp = base_temp - (self.elevations / 1000.0) * lapse_rate
    
    self.fields['temperature'] = temp
    return temp


I got rid of geo and now terrains have climates with lat lon spans in them. can you refactor comput temperature?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Sure! What's the new attribute name and structure? Is it `self.climate` with `.lat_min`, `.lat_max` etc, or something different like `self.preset`?

In [ ]:
@patch
def maui(self:TerraDemo,debug=False):
    terrain = self.mauiMap(debug=debug)
    
    #terrain.field_summary('precipitation')
        
    terrain.compute_distance_to_coast()
    if debug:
        terrain.field_summary('distance_to_coast')
    terrain.compute_temperature()  # Tropical base
    if debug:
        
        terrain.field_summary('temperature')


In [ ]:
TerraDemo().maui(True)

Grid: 100 x 70
Resolution: dx=1069m (~1.1km), dy=511m (~0.5km)
Latitude: 20.57° to 21.03° (center: 20.8°)
Wind: 8.0 m/s from 50.0°
Background precip: 0.15 mm/h

Precipitation range: 4 - 15267 mm/year
Mean: 2466 mm/year

=== DISTANCE_TO_COAST ===
Range:  0.0 to 20.0
Mean:   3.0
Median: 0.0
StdDev: 4.6
Percentiles: 10%=0.0, 25%=0.0, 75%=5.0, 90%=10.0

Distribution:
      0.0 -     2.5: ████████████████████████████████████████ (4568)
      2.5 -     5.0: ████ (528)
      5.0 -     7.5: █████ (680)
      7.5 -    10.0: ███ (398)
     10.0 -    12.5: ███ (376)
     12.5 -    15.0: █ (181)
     15.0 -    17.5: █ (201)
     17.5 -    20.0:  (68)

=== TEMPERATURE ===
Range:  -1.6 to 20.0
Mean:   15.1
Median: 15.3
StdDev: 3.7
Percentiles: 10%=10.5, 25%=14.1, 75%=17.8, 90%=19.3

Distribution:
     -1.6 -     1.1:  (32)
      1.1 -     3.8: █ (111)
      3.8 -     6.5: ██ (151)
      6.5 -     9.2: ███ (228)
      9.2 -    11.9: ███████ (439)
     11.9 -    14.6: ████████████████████████ (1526)
 

In [ ]:
#| export
@patch
def validate_coastal_temps(self: Terrain):
    """Check that coastal areas aren't getting tundra temperatures."""
    if 'distance_to_coast' not in self.fields or 'temperature' not in self.fields:
        return "Missing required fields"
    
    # Check coastal hexes (distance <= 2)
    coastal_mask = self.fields['distance_to_coast'] <= 2
    coastal_temps = self.fields['temperature'][coastal_mask]
    coastal_elevs = self.elevations[coastal_mask]
    
    print("=== COASTAL VALIDATION ===")
    print(f"Coastal hexes: {coastal_mask.sum()}")
    print(f"Temp range: {coastal_temps.min():.1f}°C to {coastal_temps.max():.1f}°C")
    print(f"Elevation range: {coastal_elevs.min():.0f}m to {coastal_elevs.max():.0f}m")
    print(f"Mean temp: {coastal_temps.mean():.1f}°C")
    
    # Check for problematic cold coastal areas
    cold_coastal = (coastal_mask) & (self.fields['temperature'] < 10)
    if cold_coastal.sum() > 0:
        print(f"\n⚠️  {cold_coastal.sum()} coastal hexes < 10°C")
        cold_elevs = self.elevations[cold_coastal]
        print(f"   These are at elevations: {cold_elevs.min():.0f}m to {cold_elevs.max():.0f}m")
        print(f"   (Likely coastal mountains - OK for biomes)")
    else:
        print(f"\n✓ No unrealistic cold coastal areas")
    
    # Check high elevation inland
    high_inland = (self.fields['distance_to_coast'] > 5) & (self.elevations > 2000)
    if high_inland.sum() > 0:
        high_temps = self.fields['temperature'][high_inland]
        print(f"\nHigh mountains (>2000m, inland):")
        print(f"  Count: {high_inland.sum()}")
        print(f"  Temp range: {high_temps.min():.1f}°C to {high_temps.max():.1f}°C")


In [ ]:
#| export
@patch
def caliRain(demo:TerraDemo):

    demo = TerraDemo()
    terrain = demo.california_with_geo()

    # Compute precipitation
    terrain.compute_precipitation_sb(
        wind_speed=10.0,
        wind_dir=270.0,
        precip_base=0.1,
        nm=0.01,
        hw=2000.0,
        cw=0.002,
        conv_time=2000.0,
        fall_time=2000.0
    )

    # Compute distance and temperature
    terrain.compute_distance_to_coast()
    terrain.field_summary('distance_to_coast')

    terrain.compute_temperature( )
    terrain.field_summary('temperature')

    # Compare to precipitation
    terrain.compare_fields('elevation', 'temperature')
    terrain.compare_fields('precipitation', 'temperature')
    terrain.validate_coastal_temps()


In [ ]:
TerraDemo().caliRain()


=== DISTANCE_TO_COAST ===
Range:  0.0 to 22.0
Mean:   5.6
Median: 4.0
StdDev: 5.9
Percentiles: 10%=0.0, 25%=0.0, 75%=9.0, 90%=15.0

Distribution:
      0.0 -     2.8: ████████████████████████████████████████ (654)
      2.8 -     5.5: █████████████████ (290)
      5.5 -     8.2: █████████████ (217)
      8.2 -    11.0: ██████ (109)
     11.0 -    13.8: ███████ (122)
     13.8 -    16.5: █████ (94)
     16.5 -    19.2: ████ (66)
     19.2 -    22.0: ██ (48)

=== TEMPERATURE ===
Range:  -6.4 to 20.0
Mean:   11.4
Median: 12.9
StdDev: 6.6
Percentiles: 10%=2.6, 25%=5.3, 75%=17.5, 90%=19.4

Distribution:
     -6.4 -    -3.1:  (10)
     -3.1 -     0.2: ██ (34)
      0.2 -     3.5: ███████████████ (197)
      3.5 -     6.8: ██████████████████████ (285)
      6.8 -    10.1: ███████████ (149)
     10.1 -    13.4: ████████████ (152)
     13.4 -    16.7: █████████████████████ (270)
     16.7 -    20.0: ████████████████████████████████████████ (503)

=== precipitation vs temperature ===
Correlatio

In [ ]:
#| export
@patch
def maui(self:TerraDemo,debug=True):
    terrain = self.mauiMap(debug=debug)
    if debug:
        terrain.field_summary('precipitation')
            
        terrain.compute_distance_to_coast()
        terrain.field_summary('distance_to_coast')
        terrain.compute_temperature()  # Tropical base
        terrain.field_summary('temperature')

        # Compare to precipitation
        terrain.compare_fields('elevation', 'temperature')
        terrain.compare_fields('precipitation', 'temperature')
        terrain.validate_coastal_temps()

In [ ]:
TerraDemo().maui()


Grid: 100 x 70
Resolution: dx=1069m (~1.1km), dy=511m (~0.5km)
Latitude: 20.57° to 21.03° (center: 20.8°)
Wind: 8.0 m/s from 50.0°
Background precip: 0.15 mm/h

Precipitation range: 4 - 15267 mm/year
Mean: 2466 mm/year

=== PRECIPITATION ===
Range:  3.8 to 15266.5
Mean:   2466.4
Median: 1801.9
StdDev: 3101.7
Percentiles: 10%=87.6, 25%=87.6, 75%=3019.4, 90%=6558.5

Distribution:
      3.8 -  1911.6: ████████████████████████████████████████ (3741)
   1911.6 -  3819.5: ████████████████████ (1908)
   3819.5 -  5727.3: █████ (512)
   5727.3 -  7635.1: ███ (291)
   7635.1 -  9543.0: █ (164)
   9543.0 - 11450.8: █ (140)
  11450.8 - 13358.7: █ (132)
  13358.7 - 15266.5: █ (112)

=== DISTANCE_TO_COAST ===
Range:  0.0 to 20.0
Mean:   3.0
Median: 0.0
StdDev: 4.6
Percentiles: 10%=0.0, 25%=0.0, 75%=5.0, 90%=10.0

Distribution:
      0.0 -     2.5: ████████████████████████████████████████ (4568)
      2.5 -     5.0: ████ (528)
      5.0 -     7.5: █████ (680)
      7.5 -    10.0: ███ (398)
     10.0

## Climates

In [ ]:
#| export
from enum import Enum

class Climate(Enum):
    MARINE = 0        # Ocean/sea
    FRESHWATER = 1    # Lakes/rivers
    TUNDRA = 2        # Cold, low precipitation
    DESERT = 3        # Hot/cold, very low precipitation
    GRASSLAND = 4     # Moderate temp, moderate precipitation
    FOREST = 5        # Moderate temp, high precipitation
    JUNGLE = 6        # Hot, very high precipitation







In [ ]:
#| export
@patch
def configure(preset:ClimatePreset,terrain: Terrain, force_recompute: bool = False,debug=False):
    """
    Apply the stored climate preset to compute precipitation, temperature, and climate zones.
    
    Args:
        terrain: Terrain with climate_preset set
        force_recompute: Recompute even if fields already exist
    """
   
    
    # Compute precipitation
    if 'precipitation' not in terrain.fields or force_recompute:
        if debug:
            print("\nComputing precipitation...")
        terrain.compute_precipitation_sb(
            wind_speed=preset.wind_speed,
            wind_dir=preset.wind_dir,
            precip_base=preset.precip_base,
            nm=preset.nm,
            hw=preset.hw,
            cw=preset.cw,
            conv_time=preset.conv_time,
            fall_time=preset.fall_time
        )
    
    # Compute distance to coast
    if 'distance_to_coast' not in terrain.fields or force_recompute:
        if debug:
            print("Computing distance to coast...")
        terrain.compute_distance_to_coast()
    
    # Compute temperature
    if 'temperature' not in terrain.fields or force_recompute:
        if debug:
            print("Computing temperature...")
        terrain.compute_temperature(
            base_temp_south=preset.base_temp_range[0],
            base_temp_north=preset.base_temp_range[1],
            lapse_rate=preset.lapse_rate
        )
    
    # Compute climate zones
    if 'climate' not in terrain.fields or force_recompute:
        if debug:
            print("Computing climate zones...")
        terrain.compute_climate()
        if debug:
            terrain.summarize_climate()


In [ ]:
#| export


@patch
def summarize_climate(self: Terrain):
    """Print summary statistics of climate distribution."""
    if 'climate' not in self.fields:
        print("No climate data - run compute_climate() first")
        return
    
    climate = self.fields['climate']
    total = len(climate)
    
    print("\n=== CLIMATE DISTRIBUTION ===")
    print("-" * 40)
    for climate_type in Climate:
        count = np.sum(climate == climate_type.value)
        pct = 100 * count / total
        bar = '█' * int(30 * count / total)
        print(f"{climate_type.name:12s}: {bar} {count:4d} ({pct:5.1f}%)")
    
    print("\n=== FIELD STATS ===")
    if 'temperature' in self.fields:
        t = self.fields['temperature']
        print(f"Temperature: {t.min():.1f}°C to {t.max():.1f}°C (mean: {t.mean():.1f}°C)")
    
    if 'precipitation' in self.fields:
        p = self.fields['precipitation']
        print(f"Precipitation: {p.min():.0f}mm to {p.max():.0f}mm (mean: {p.mean():.0f}mm)")


In [ ]:
#| export
## pass 2
@patch
def compute_climate(self: Terrain):
    """Classify climate zones based on temperature and precipitation."""
    n_hexes = len(self.elevations)
    
    if 'temperature' not in self.fields:
        raise ValueError("Must compute temperature first")
    if 'precipitation' not in self.fields:
        raise ValueError("Must compute precipitation first")
    
    climate = np.zeros(n_hexes, dtype=int)
    temp = self.fields['temperature']
    precip = self.fields['precipitation']
    
    for i in range(n_hexes):
        elev = self.elevations[i]
        t = temp[i]
        p = precip[i]
        
        # Marine (ocean/sea)
        if elev <= 0:
            climate[i] = Climate.MARINE.value
        
        # Tundra (very cold) - stricter threshold
        elif t < 0:  # Changed from 5 to 0
            climate[i] = Climate.TUNDRA.value
        
        # Desert (very dry)
        elif p < 250:
            climate[i] = Climate.DESERT.value
        
        # Jungle (hot and very wet)
        elif t > 20 and p > 1500:
            climate[i] = Climate.JUNGLE.value
        
        # Forest (good rainfall)
        elif p > 750:  # Changed from 800
            climate[i] = Climate.FOREST.value
        
        # Grassland (moderate conditions)
        elif p >= 250:
            climate[i] = Climate.GRASSLAND.value
        
        # Fallback to desert
        else:
            climate[i] = Climate.DESERT.value
    
    self.fields['climate'] = climate
    return climate


In [ ]:
#| export
@patch
def add_climate_overlay(self: Terrain, layer_name="climate"):
    """Visualize climate zones with appropriate colors."""
    opacity = 0.5
    
    climate_colors = {
        Climate.MARINE.value: "#1e88e5",
        Climate.FRESHWATER.value: "#42a5f5",
        Climate.TUNDRA.value: "#e3f2fd",
        Climate.DESERT.value: "#fdd835",
        Climate.GRASSLAND.value: "#9ccc65",
        Climate.FOREST.value: "#2e7d32",
        Climate.JUNGLE.value: "#1b5e20",
    }
    
    if 'climate' not in self.fields:
        self.compute_climate()
    
    climate_indices = self.fields['climate'].astype(int)
    
    # Create patterns
    patternGen = TerrainPatterns(self)
    colors = [climate_colors[i] for i in range(len(Climate))]
    patterns = patternGen.ballDensity(len(colors), fills=colors, prefix="climate")
    
     # Add opacity to pattern styles
    for pattern in patterns:
        pattern.opacity = opacity  # This may need adjustment based on your StyleCSS implementation
    
    # Generate overlay for all hexes
    overlay = self.makeOverlay(climate_indices, patterns)
    
    self.builder.adjust(layer_name, overlay)


@patch
def downsample_climate(self: Terrain, sample_radius=2):
    """Downsample terrain with all climate data preserved.
    
    This intelligently handles:
    - Elevation (max to preserve peaks)
    - Temperature (weighted average)
    - Precipitation (weighted average)
    - Distance to coast (min - closest coast matters)
    - Climate zones (mode - most common type)
    - Geographic coordinates (weighted average)
    """
    old_grid = self.hexGrid
    factor = sample_radius + 1
    new_rows = old_grid.nRows // factor
    new_cols = old_grid.nCols // factor
    
    if new_rows < 2 or new_cols < 2:
        print("Warning: sample_radius too large for this terrain")
        return self
    
    # Create new terrain with larger hexes
    new_radius = old_grid.radius * factor
    new_terrain = Terrain(old_grid.bounds, radius=new_radius, climate=self.climate)
    
    # Copy style properties
    new_terrain.colorLevels = self.colorLevels.copy() if self.colorLevels else None
    new_terrain.elevationDelta = self.elevationDelta
    new_terrain.seaLevel = self.seaLevel
    #new_terrain.climate = self.climate
    
    # Downsample elevation (use max to preserve peaks)
    new_terrain.elevations = self.downsample_field(self.elevations, sample_radius, 'max')
    
    # Downsample all fields with appropriate methods
    field_methods = {
        'temperature': 'weighted_avg',
        'precipitation': 'weighted_avg',
        'precip_rate_mmh': 'weighted_avg',
        'distance_to_coast': 'min',  # Closest coast is what matters
        'latitude': 'weighted_avg',
        'longitude': 'weighted_avg',
        'climate': 'mode',  # Most common climate type
        'temp_range': 'weighted_avg',  # If you add this later
    }
    
    for field_name, method in field_methods.items():
        if field_name in self.fields:
            new_terrain.fields[field_name] = self.downsample_field(
                self.fields[field_name], sample_radius, method)
    
    # Add color styles
    if new_terrain.colorLevels:
        for color in new_terrain.colorLevels:
            new_terrain.hexGrid.builder.add_style(color)
    
    return new_terrain


In [ ]:
#| export
@patch
def downsample_climate(self: Terrain, scale=0.5,sample_radius=1):
    """Downsample terrain with all climate data preserved.
    
    This intelligently handles:
    - Elevation (max to preserve peaks)
    - Temperature (weighted average)
    - Precipitation (weighted average)
    - Distance to coast (min - closest coast matters)
    - Climate zones (mode - most common type)
    - Geographic coordinates (weighted average)
    
    Args:
        scale: Fraction of original size (0.5 = half size, 0.33 = third size)
    """
    
    # Helper to get hex pattern for sampling
    def hexes_in_range(n):
        """Finding all hexes within distance N from origin"""
        results = []
        for q in range(-n, n + 1):
            for r in range(max(-n, -q - n), min(n, -q + n) + 1):
                s = -q - r
                results.append(HexPosition(q, r, s))
        return results
    
    ring_pattern = hexes_in_range(sample_radius)
    
    # Start with elevation using max (preserve peaks)
    new_elevations, nRows, nCols = self.convolution(
        self.elevations, ring_pattern, scale, method='max')
    
    # Create new terrain with downsampled grid
    grid = HexGrid(
        nRows=nRows,
        nCols=nCols,
        radius=self.hexGrid.radius,
        style=self.hexGrid.style
    )
    
    mySize = MapSize(400, 400)  # These get dropped
    myBounds = MapRect(MapCord(0, 0), mySize)
    new_terrain = Terrain(myBounds, radius=15)
    new_terrain.hexGrid = grid
    new_terrain.elevations = new_elevations
    
    # Copy style properties
    new_terrain.colorLevels = self.colorLevels.copy() if self.colorLevels else None
    new_terrain.elevationDelta = self.elevationDelta
    new_terrain.seaLevel = self.seaLevel
    new_terrain.climate = self.climate
    
    # Downsample all fields with appropriate methods
    field_methods = {
        'temperature': 'weighted_avg',
        'precipitation': 'weighted_avg',
        'precip_rate_mmh': 'weighted_avg',
        'distance_to_coast': 'min',  # Closest coast matters
        'latitude': 'weighted_avg',
        'longitude': 'weighted_avg',
        'climate': 'mode',  # Most common climate type
        'temp_range': 'weighted_avg',
    }
    
    for field_name, method in field_methods.items():
        if field_name in self.fields:
            downsampled, _, _ = self.convolution(
                self.fields[field_name], ring_pattern, scale, method=method)
            new_terrain.fields[field_name] = downsampled
    
    # Add color styles
    if new_terrain.colorLevels:
        for color in new_terrain.colorLevels:
            new_terrain.hexGrid.builder.add_style(color)
    
    return new_terrain


we need to rewrite def downsample_climate(self: Terrain, sample_radius=2) since we no longer have downsample_field. the scale function as well as convolution work well for this

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at the code, I can see that `convolution` and `scaled` are the new approach. Let me refactor `downsample_climate` to use these:

```python
@patch
def downsample_climate(self: Terrain, scale=0.5):
    """Downsample terrain with all climate data preserved.
    
    This intelligently handles:
    - Elevation (max to preserve peaks)
    - Temperature (weighted average)
    - Precipitation (weighted average)
    - Distance to coast (min - closest coast matters)
    - Climate zones (mode - most common type)
    - Geographic coordinates (weighted average)
    
    Args:
        scale: Fraction of original size (0.5 = half size, 0.33 = third size)
    """
    
    # Helper to get hex pattern for sampling
    def hexes_in_range(n):
        """Finding all hexes within distance N from origin"""
        results = []
        for q in range(-n, n + 1):
            for r in range(max(-n, -q - n), min(n, -q + n) + 1):
                s = -q - r
                results.append(HexPosition(q, r, s))
        return results
    
    ring_pattern = hexes_in_range(1)
    
    # Start with elevation using max (preserve peaks)
    new_elevations, nRows, nCols = self.convolution(
        self.elevations, ring_pattern, scale, method='max')
    
    # Create new terrain with downsampled grid
    grid = HexGrid(
        nRows=nRows,
        nCols=nCols,
        radius=self.hexGrid.radius,
        style=self.hexGrid.style
    )
    
    mySize = MapSize(400, 400)  # These get dropped
    myBounds = MapRect(MapCord(0, 0), mySize)
    new_terrain = Terrain(myBounds, radius=15)
    new_terrain.hexGrid = grid
    new_terrain.elevations = new_elevations
    
    # Copy style properties
    new_terrain.colorLevels = self.colorLevels.copy() if self.colorLevels else None
    new_terrain.elevationDelta = self.elevationDelta
    new_terrain.seaLevel = self.seaLevel
    new_terrain.climate = self.climate
    
    # Downsample all fields with appropriate methods
    field_methods = {
        'temperature': 'weighted_avg',
        'precipitation': 'weighted_avg',
        'precip_rate_mmh': 'weighted_avg',
        'distance_to_coast': 'min',  # Closest coast matters
        'latitude': 'weighted_avg',
        'longitude': 'weighted_avg',
        'climate': 'mode',  # Most common climate type
        'temp_range': 'weighted_avg',
    }
    
    for field_name, method in field_methods.items():
        if field_name in self.fields:
            downsampled, _, _ = self.convolution(
                self.fields[field_name], ring_pattern, scale, method=method)
            new_terrain.fields[field_name] = downsampled
    
    # Add color styles
    if new_terrain.colorLevels:
        for color in new_terrain.colorLevels:
            new_terrain.hexGrid.builder.add_style(color)
    
    return new_terrain
```

The key changes:
1. Uses `convolution` instead of `downsample_field`
2. Uses `scale` parameter (fraction) instead of `sample_radius`
3. Manually constructs the new grid like `scaled` does
4. Applies different methods for different fields (max for elevation, mode for climate, etc.)

In [ ]:
@patch
def caliClimate(self:TerraDemo,debug=True):
    terrain_ca = self.california_with_geo()
    
    terrain_ca.climate = ClimatePreset(
    name='California',
    lat_range=(32.5, 42.0),  # Your actual CA bounds
    base_temp_range=(20, 14),  # Warmer south, cooler north
    wind_speed=10.0,
    wind_dir=270.0,  # Pacific winds from west
    precip_base=0.1,
    nm=0.01,
    hw=2000.0,
    cw=0.002,
    conv_time=2000.0,
    fall_time=2000.0,
    lapse_rate=6.5,
    precip_bins=(200, 500, 1000, 2000),  # Desert to rainforest range
    description="Mediterranean with extreme orographic effects. Mojave to redwoods."
)
    terrain_ca.hexGrid.adjustRadius(15)
    terrain_ca.climate.configure(terrain_ca,debug=debug)

    terrain_ca.compute_distance_to_coast()
    terrain_ca.compute_temperature(base_temp_south=20.0, base_temp_north=14.0, lapse_rate=6.5)
    terrain_ca.compute_climate()
    smaller = terrain_ca.downsample_climate(sample_radius=2)
    #smaller = terrain_ca
    
    smaller.colorMap()
    smaller.hexGrid.update()

    if debug:
        smaller.summarize_climate()
    # Visualize the downsampled terrain
    smaller.add_climate_overlay()
    return smaller.hexGrid.builder.show()

In [ ]:
TerraDemo().caliClimate(False)

### Pretty pictures

In [ ]:
#| export
class WeatherColor:
    """Color scheme for weather map."""

    def __init__(self,color: str = "#000000"):
        self.color = color  # <-- Need the assignment!
        self.r, self.g, self.b = int(color[1:3], 16), int(color[3:5], 16), int(color[5:7], 16)
        self.h, self.l, self.s = colorsys.rgb_to_hls(self.r/255, self.g/255, self.b/255)

    def __str__(self):
        return self.color

    def __repr__(self):
        return f"r:{self.r:02} g:{self.g:02} b:{self.b:02}   "

    
    @staticmethod
    def temp_to_color(temp: float) -> 'WeatherColor':
        """Map temperature to smooth weather map gradient (purple → blue → green → yellow → red)."""
  
        
        # Define color stops: (temperature, hex_color)
        stops = [
            (-15, "#8B4789"),  # Purple (very cold)
            (0,   "#4A90E2"),  # Blue (freezing)
            (10,  "#50C878"),  # Green (cool)
            (20,  "#F4D03F"),  # Yellow (warm)
            (30,  "#E74C3C"),  # Red (hot)
        ]
        
        # Clamp temperature to range
        if temp <= stops[0][0]:
            return stops[0][1]
        if temp >= stops[-1][0]:
            return stops[-1][1]
        
        # Find the two stops to interpolate between
        for i in range(len(stops) - 1):
            temp1, color1 = stops[i]
            temp2, color2 = stops[i + 1]
            
            if temp1 <= temp <= temp2:
                # Calculate interpolation factor (0 to 1)
                factor = (temp - temp1) / (temp2 - temp1)
                
                # Convert hex to RGB
                r1, g1, b1 = int(color1[1:3], 16), int(color1[3:5], 16), int(color1[5:7], 16)
                r2, g2, b2 = int(color2[1:3], 16), int(color2[3:5], 16), int(color2[5:7], 16)
                
                # Interpolate in RGB space
                r = int(r1 + (r2 - r1) * factor)
                g = int(g1 + (g2 - g1) * factor)
                b = int(b1 + (b2 - b1) * factor)
                
                return WeatherColor(f"#{r:02x}{g:02x}{b:02x}")
        
        return WeatherColor("#808080" ) # Fallback gray
    
   
    def adjust_brightness(self, elevation: float, max_elev: float = 3000.0, range_pct: float = 0.10) -> 'WeatherColor':
        """Adjust color brightness based on elevation.
        
        Args:
            color: Hex color string
            elevation: Elevation in meters
            max_elev: Maximum elevation for normalization
            range_pct: Brightness adjustment range (0.10 = ±10%)
        """
        import colorsys
        color = self.color
        
        # Normalize elevation to [-1, 1] range
        elev_norm = np.clip(elevation / max_elev, 0, 1) * 2 - 1
        
        
        
        # Adjust lightness: higher elevation = brighter (within range_pct)
        l = self.l + (elev_norm * range_pct)
        l = np.clip(l, 0, 1)
        
        # Convert back to RGB
        r, g, b = colorsys.hls_to_rgb(self.h, l, self.s)
        
        return  WeatherColor(f"#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}")
    
    
    def adjust_saturation(self, precip: float, climate_preset: ClimatePreset) -> 'WeatherColor':
        """Adjust color saturation based on precipitation.
        
        Args:
            color: Hex color string
            precip: Precipitation in mm/year
            climate_preset: ClimatePreset with precip_bins
        """
        color = self.color
        
        # Normalize precipitation using climate's bins
        bins = climate_preset.precip_bins
        if precip <= bins[0]:
            sat_factor = 0.5  # Very dry = 50% saturation
        elif precip >= bins[-1]:
            sat_factor = 1.0  # Very wet = 100% saturation
        else:
            # Linear interpolation between bins
            for i in range(len(bins) - 1):
                if bins[i] <= precip <= bins[i+1]:
                    sat_factor = 0.5 + 0.5 * ((precip - bins[i]) / (bins[i+1] - bins[i]))
                    break
        
       
        
        # Adjust saturation
        s = self.s * sat_factor
        s = np.clip(s, 0, 1)
        
        # Convert back to RGB
        r, g, b = colorsys.hls_to_rgb(self.h, self.l, s)
        
        return WeatherColor(f"#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}")

@patch
def render_icon_temperature(self: Terrain, ocean_color: str = "#2c5f8d") -> str:
    """Render hexes with inline fill based on temperature gradient.
    
    Args:
        ocean_color: Solid color for ocean hexes
    
    Returns:
        SVG string with solid-colored hexes
    """
    if 'temperature' not in self.fields:
        raise ValueError("Temperature field not computed")
    
    if 'precipitation' not in self.fields:
        raise ValueError("Precipitation field not computed")
    
    overlay = ""
    grid = self.hexGrid
    
    # Find max elevation for normalization
    land_elevs = self.elevations[self.elevations > 0]
    max_elev = land_elevs.max() if len(land_elevs) > 0 else 3000.0
    
    for i in range(len(self.elevations)):
        if self.elevations[i] <= 0:
            # Ocean
            color = ocean_color
        else:
            # Land: temperature gradient
            temp = self.fields['temperature'][i]
            color = WeatherColor.temp_to_color(temp)
            
            # Modulate by elevation (brightness)
            color = color.adjust_brightness( self.elevations[i], max_elev)
            
            # Modulate by precipitation (saturation)
            if self.climate:
                color = color.adjust_saturation( self.fields['precipitation'][i], self.climate)
        
        # Generate polygon with inline fill
        hex_obj = grid.hexes[i]
        points = " ".join([f"{p.x:.0f},{p.y:.0f}" for p in hex_obj.vertices()])
        overlay += f'\t<polygon points="{points}" fill="{color}"/>\n'
    
    return overlay

In [ ]:
#| export
@patch
def gradient_temperature(self: Terrain, ocean_color: str = "#2c5f8d") -> str:
    """Render hexes with inline fill based on temperature gradient.
    
    Args:
        ocean_color: Solid color for ocean hexes
    
    Returns:
        SVG string with solid-colored hexes
    """
    if 'temperature' not in self.fields:
        raise ValueError("Temperature field not computed")
    
    if 'precipitation' not in self.fields:
        raise ValueError("Precipitation field not computed")
    
    overlay = ""
    grid = self.hexGrid
    
    # Find max elevation for normalization
    land_elevs = self.elevations[self.elevations > 0]
    max_elev = land_elevs.max() if len(land_elevs) > 0 else 3000.0
    lookUp = {}
    
    for i in range(len(self.elevations)):
        if self.elevations[i] <= 0:
            # Ocean
            color = ocean_color
        else:
            # Land: temperature gradient
            temp = self.fields['temperature'][i]
            color = WeatherColor.temp_to_color(temp)
            
            # Modulate by elevation (brightness)
            color = color.adjust_brightness( self.elevations[i], max_elev)
            
            # Modulate by precipitation (saturation)
            if self.climate:
                color = color.adjust_saturation( self.fields['precipitation'][i], self.climate)
        
        # Generate polygon with inline fill
        lookUp[i] = color
    overlay = self.hexGrid.radial_gradient(lookUp)
    
    return overlay


In [ ]:
#| export
@patch
def makeClimateOverlay(self: Terrain, climate_data, precip_data, all_patterns: list[SVGBuilder]) -> str:
    """
    Create overlay for climate+precipitation visualization.
    
    Args:
        climate_data: array of climate type values (0-6)
        precip_data: array of precipitation values (mm/year)
        all_patterns: list of 35 patterns (7 climates * 5 precip levels)
    
    Returns:
        SVG string for the overlay
    """
    testBody = ""
    grid = self.hexGrid
    preset = self.climate
    
    # Track which patterns are actually used
    used_patterns = set()
    
    for i in range(len(climate_data)):
        climate_val = int(climate_data[i])
        precip = precip_data[i]
        precip_level = preset.get_precip_level(precip)
        
        # Calculate flat pattern index
        pattern_idx = climate_val * 5 + precip_level
        
        # Bounds check
        if pattern_idx >= 0 and pattern_idx < len(all_patterns):
            used_patterns.add(pattern_idx)
            
            patName = all_patterns[pattern_idx].attributes['id']
            fill = f"url(#{patName})"
            
            ret = "<polygon points=\""
            hex = grid.hexes[i]
            for point in hex.vertices():
                ret += f"{point.x:.0f},{point.y:.0f} "
            ret += f"\" style=\"fill:{fill}\""
            ret += "/>"
            
            testBody += "\t" + ret + "\n"
    
    # Only add used patterns to builder
    for pattern_idx in sorted(used_patterns):
        grid.builder.add_definition(all_patterns[pattern_idx])
    
    return testBody

In [ ]:
#| export
@patch
def dottedClimate(self: Terrain, 
                  flow_levels: int = 5,
                  min_density: float = 0.25,  # Minimum dot density for dry areas
                  debug: bool = False) -> str:
    """Combine climate colors with precipitation-based dot density.
    
    Args:
        flow_levels: Number of dot density levels (default 5)
        min_density: Minimum density for low-precip areas (0.25 = 25%)
        debug: Print pattern generation info
    
    Returns:
        SVG string for the overlay
    """
    
    if 'climate' not in self.fields or 'precipitation' not in self.fields:
        raise ValueError("Need both climate and precipitation computed")
    
    # Define climate colors (muted palette)
    climate_colors = {
        Climate.MARINE: "#A9B9D4",
        Climate.FRESHWATER: "#7BA3C0",
        Climate.TUNDRA: "#8E7159",
        Climate.DESERT: "#D8A48F",
        Climate.GRASSLAND: "#B9B291",
        Climate.FOREST: "#7A9B76",
        Climate.JUNGLE: "#5D7C5A",
    }
    
    # Generate patterns for each climate type
    patternGen = TerrainPatterns(self)
    all_patterns = []
    
    for climate_type in Climate:
        color = climate_colors.get(climate_type, "#cccccc")
        
        # Create flow_levels density patterns for this climate color
        patterns = patternGen.ballDensity(
            levels=flow_levels + 3,
            fills=[color],
            prefix=f"{climate_type.name.lower()}_ball"
        )[:flow_levels]
        
        if debug:
            print(f"{climate_type.name}: generated {len(patterns)} patterns with color {color}")
        
        all_patterns.extend(patterns)
    
    # Build overlay
    grid = self.hexGrid
    overlay = ""
    used_patterns = set()
    
    climate_data = self.fields['climate']
    precip_data = self.fields['precipitation']
    
    # Find max precipitation for normalization
    max_precip = precip_data.max()
    min_precip = precip_data.min()
    
    if debug:
        print(f"\nPrecipitation range: {min_precip:.0f} - {max_precip:.0f} mm/year")
        print(f"Min density: {min_density * 100}%")
    
    # Generate hex polygons
    for i in range(len(climate_data)):
        climate_val = int(climate_data[i])
        precip = precip_data[i]
        
        # Normalize precipitation to [min_density, 1.0] range
        if max_precip > min_precip:
            normalized_precip = (precip - min_precip) / (max_precip - min_precip)  # [0, 1]
            # Scale to [min_density, 1.0]
            scaled_precip = min_density + normalized_precip * (1.0 - min_density)
            precip_level = int(scaled_precip * (flow_levels - 1))
            precip_level = min(precip_level, flow_levels - 1)
        else:
            precip_level = 0
        
        # Calculate pattern index: climate_val * flow_levels + precip_level
        pattern_idx = climate_val * flow_levels + precip_level
        
        # Bounds check
        if pattern_idx >= 0 and pattern_idx < len(all_patterns):
            used_patterns.add(pattern_idx)
            
            patName = all_patterns[pattern_idx].attributes['id']
            fill = f"url(#{patName})"
            
            # Generate polygon
            hex_obj = grid.hexes[i]
            ret = "<polygon points=\""
            for point in hex_obj.vertices():
                ret += f"{point.x:.0f},{point.y:.0f} "
            ret += f"\" style=\"fill:{fill}\"/>"
            
            overlay += "\t" + ret + "\n"
    
    # Add used patterns to builder
    for pattern_idx in sorted(used_patterns):
        grid.builder.add_definition(all_patterns[pattern_idx])
    
    if debug:
        print(f"\nUsed {len(used_patterns)} patterns out of {len(all_patterns)}")
    
    return overlay


In [ ]:
#| export
@patch
def add_climate_overlay(self: Terrain, layer_name="climate_precip",debug=False):
    """Combine climate colors with precipitation-based dot density."""
   
    overlay =  self.dottedClimate(debug=debug)
    self.hexGrid.builder.adjust(layer_name, overlay)
    
    return self

In [ ]:
@patch
def caliNice(self:TerraDemo,debug=True):
    terrain_ca = self.california_with_geo()
    
    terrain_ca.hexGrid.adjustRadius(15)
    terrain_ca.climate = ClimatePreset(
    name='California',
    lat_range=(32.5, 42.0),  # Your actual CA bounds
    base_temp_range=(20, 14),  # Warmer south, cooler north
    wind_speed=10.0,
    wind_dir=270.0,  # Pacific winds from west
    precip_base=0.1,
    nm=0.01,
    hw=2000.0,
    cw=0.002,
    conv_time=2000.0,
    fall_time=2000.0,
    lapse_rate=6.5,
    precip_bins=(200, 500, 1000, 2000),  # Desert to rainforest range
    description="Mediterranean with extreme orographic effects. Mojave to redwoods."
)
    terrain_ca.climate.configure(terrain_ca,debug=debug)
    terrain_ca.compute_climate()

    smaller = terrain_ca.downsample_climate(sample_radius=2)
    smaller.hexGrid.adjustRadius(25)
    #smaller = terrain_ca
    
    smaller.colorMap()
    smaller.hexGrid.update()

    if debug:
        smaller.summarize_climate()
    # Visualize the downsampled terrain
    #smaller.add_climate_overlay()
        # Add the climate+precipitation overlay
    smaller.add_climate_overlay(layer_name="climate_precip")
    smaller.add_elevation_borders()
    
    # Clear other layers for clean view
    smaller.hexGrid.builder.adjust("root", "")
    smaller.hexGrid.builder.adjust("climate", "")
    return smaller.hexGrid.builder.show()

In [ ]:
TerraDemo().caliNice()


Computing precipitation...
Computing distance to coast...
Computing temperature...
Computing climate zones...

=== CLIMATE DISTRIBUTION ===
----------------------------------------
MARINE      : ████████  460 ( 28.8%)
FRESHWATER  :     0 (  0.0%)
TUNDRA      :    41 (  2.6%)
DESERT      : █████████  516 ( 32.2%)
GRASSLAND   : ██  107 (  6.7%)
FOREST      : ████████  476 ( 29.8%)
JUNGLE      :     0 (  0.0%)

=== FIELD STATS ===
Temperature: -6.4°C to 20.0°C (mean: 11.4°C)
Precipitation: 7mm to 10361mm (mean: 1310mm)



=== CLIMATE DISTRIBUTION ===
----------------------------------------
MARINE      : ████████  116 ( 29.0%)
FRESHWATER  :     0 (  0.0%)
TUNDRA      :     1 (  0.2%)
DESERT      : ███████████  153 ( 38.2%)
GRASSLAND   :     1 (  0.2%)
FOREST      : █████████  129 ( 32.2%)
JUNGLE      :     0 (  0.0%)

=== FIELD STATS ===
Temperature: 0.7°C to 19.8°C (mean: 11.4°C)
Precipitation: 84mm to 4842mm (mean: 1338mm)


In [ ]:
@patch
def caliNice(self:TerraDemo,debug=True):
    terrain_ca = self.california_with_geo()
    
    terrain_ca.hexGrid.adjustRadius(15)
    terrain_ca.climate = ClimatePreset(
    name='California',
    lat_range=(32.5, 42.0),  # Your actual CA bounds
    base_temp_range=(20, 14),  # Warmer south, cooler north
    wind_speed=10.0,
    wind_dir=270.0,  # Pacific winds from west
    precip_base=0.1,
    nm=0.01,
    hw=2000.0,
    cw=0.002,
    conv_time=2000.0,
    fall_time=2000.0,
    lapse_rate=6.5,
    precip_bins=(200, 500, 1000, 2000),  # Desert to rainforest range
    description="Mediterranean with extreme orographic effects. Mojave to redwoods."
)
    terrain_ca.climate.configure(terrain_ca,debug=debug)
    terrain_ca.compute_climate()

    smaller = terrain_ca.downsample_climate(sample_radius=2)
    smaller.hexGrid.adjustRadius(25)
    #smaller = terrain_ca
    
    smaller.colorMap()
    smaller.hexGrid.update()

    if debug:
        smaller.summarize_climate()
    # Visualize the downsampled terrain
    #smaller.add_climate_overlay()
        # Add the climate+precipitation overlay
    smaller.add_climate_overlay(layer_name="climate_precip")
    smaller.add_elevation_borders()
    
    # Clear other layers for clean view
    smaller.hexGrid.builder.adjust("root", "")
    smaller.hexGrid.builder.adjust("climate", "")
   
    terrb = smaller.hexGrid.builder

    # Clear all existing layers
    for l in terrb.layers:
        terrb.adjust(l.name, "")

# Render temperature-based weather map
    ov = smaller.render_icon_temperature()
    terrb.adjust("weather", ov)  # Fixed: agjust → adjust

    return terrb.show()


In [ ]:
TerraDemo().caliNice()


Computing precipitation...
Computing distance to coast...
Computing temperature...
Computing climate zones...

=== CLIMATE DISTRIBUTION ===
----------------------------------------
MARINE      : ████████  460 ( 28.8%)
FRESHWATER  :     0 (  0.0%)
TUNDRA      :    41 (  2.6%)
DESERT      : █████████  516 ( 32.2%)
GRASSLAND   : ██  107 (  6.7%)
FOREST      : ████████  476 ( 29.8%)
JUNGLE      :     0 (  0.0%)

=== FIELD STATS ===
Temperature: -6.4°C to 20.0°C (mean: 11.4°C)
Precipitation: 7mm to 10361mm (mean: 1310mm)



=== CLIMATE DISTRIBUTION ===
----------------------------------------
MARINE      : ████████  116 ( 29.0%)
FRESHWATER  :     0 (  0.0%)
TUNDRA      :     1 (  0.2%)
DESERT      : ███████████  153 ( 38.2%)
GRASSLAND   :     1 (  0.2%)
FOREST      : █████████  129 ( 32.2%)
JUNGLE      :     0 (  0.0%)

=== FIELD STATS ===
Temperature: 0.7°C to 19.8°C (mean: 11.4°C)
Precipitation: 84mm to 4842mm (mean: 1338mm)


In [ ]:


@patch
def caliNice(self:TerraDemo,debug=True):
    terrain_ca = self.california_with_geo()
    
    terrain_ca.hexGrid.adjustRadius(15)
    terrain_ca.climate = ClimatePreset(
    name='California',
    lat_range=(32.5, 42.0),  # Your actual CA bounds
    base_temp_range=(20, 14),  # Warmer south, cooler north
    wind_speed=10.0,
    wind_dir=270.0,  # Pacific winds from west
    precip_base=0.1,
    nm=0.01,
    hw=2000.0,
    cw=0.002,
    conv_time=2000.0,
    fall_time=2000.0,
    lapse_rate=6.5,
    precip_bins=(200, 500, 1000, 2000),  # Desert to rainforest range
    description="Mediterranean with extreme orographic effects. Mojave to redwoods."
)
    terrain_ca.climate.configure(terrain_ca,debug=debug)
    terrain_ca.compute_climate()

    smaller = terrain_ca.downsample_climate(sample_radius=2)
    smaller.hexGrid.adjustRadius(25)
    #smaller = terrain_ca
    
    smaller.colorMap()
    smaller.hexGrid.update()

    if debug:
        smaller.summarize_climate()
    # Visualize the downsampled terrain
    #smaller.add_climate_overlay()
        # Add the climate+precipitation overlay
    smaller.add_climate_overlay(layer_name="climate_precip")
    smaller.add_elevation_borders()
    
    # Clear other layers for clean view
    smaller.hexGrid.builder.adjust("root", "")
    smaller.hexGrid.builder.adjust("climate", "")
   
    terrb = smaller.hexGrid.builder

    # Clear all existing layers
    for l in terrb.layers:
        terrb.adjust(l.name, "")

# Render temperature-based weather map
    ov = smaller.gradient_temperature()
    terrb.adjust("weather", ov)  # Fixed: agjust → adjust

    return terrb.show()



In [ ]:


TerraDemo().caliNice()




Computing precipitation...
Computing distance to coast...
Computing temperature...
Computing climate zones...

=== CLIMATE DISTRIBUTION ===
----------------------------------------
MARINE      : ████████  460 ( 28.8%)
FRESHWATER  :     0 (  0.0%)
TUNDRA      :    41 (  2.6%)
DESERT      : █████████  516 ( 32.2%)
GRASSLAND   : ██  107 (  6.7%)
FOREST      : ████████  476 ( 29.8%)
JUNGLE      :     0 (  0.0%)

=== FIELD STATS ===
Temperature: -6.4°C to 20.0°C (mean: 11.4°C)
Precipitation: 7mm to 10361mm (mean: 1310mm)

=== CLIMATE DISTRIBUTION ===
----------------------------------------
MARINE      : ████████  116 ( 29.0%)
FRESHWATER  :     0 (  0.0%)
TUNDRA      :     1 (  0.2%)
DESERT      : ███████████  153 ( 38.2%)
GRASSLAND   :     1 (  0.2%)
FOREST      : █████████  129 ( 32.2%)
JUNGLE      :     0 (  0.0%)

=== FIELD STATS ===
Temperature: 0.7°C to 19.8°C (mean: 11.4°C)
Precipitation: 84mm to 4842mm (mean: 1338mm)


### take me to the river



## Central Valley River Generation - Summary

### Overview
We built a rainfall-based river generation system that creates rivers flowing from the Sierra Nevada mountains through the Central Valley to the ocean. Rivers start at high-elevation sources with sufficient rainfall and flow downhill following terrain gradients.

### Key Methods

**`generate_central_valley_rivers(min_rainfall=400, min_elevation=800, valley_floor=20)`**
- Finds river sources in Sierra Nevada (high elevation + rainfall)
- Creates rivers from each source using `create_valley_river()`
- Merges intersecting rivers with `River.combine_rivers()`
- Sorts by accumulated rainfall (proxy for river importance)
- Returns list of `River` objects

**`create_valley_river(source_hex, valley_floor=20)`**
- Traces path downhill from source using `lowest_neighbor()`
- Accumulates rainfall along path (used for ranking)
- Stops at ocean (elevation < 1) or when stuck
- Tracks visited hexes to prevent loops
- Returns `River` with path stored in tree structure

### Parameters
| Parameter | Default | Purpose |
|-----------|---------|---------|
| `min_rainfall` | 400mm | Minimum rainfall at source |
| `min_elevation` | 800m | Minimum elevation for sources |
| `valley_floor` | 20m | Threshold for valley continuation |

### Visualization
- Top rivers styled by importance (thicker = more accumulated rainfall)
- `river0`: Sacramento/San Joaquin (stroke-width 8, dark blue)
- `river1`: Major tributaries (stroke-width 5)
- `river2`: Minor tributaries (stroke-width 3)

### Known Issues to Address
- Sacramento and San Joaquin rivers not appearing as expected
- Rivers stopping too early (may need lower `valley_floor` or different stop conditions)
- May need to add explicit drainage basin logic to ensure rivers converge properly in valley
- Consider adding flow accumulation algorithm for more realistic river networks

### Potential Improvements
1. **Flow accumulation**: Sum upstream contributing area rather than just rainfall
2. **Explicit outlets**: Define San Francisco Bay area as the target for valley rivers
3. **Valley routing**: Special handling for flat valley floor where gradient-following fails
4. **River merging**: Better confluence detection where tributaries join main rivers

In [ ]:
#| export
@patch  
def show(self: River, builder: SVGBuilder, layer: str = "river", 
         style: dict = None):
    """Display river on SVG builder.
    
    Args:
        builder: SVGBuilder to add river to
        layer: Layer name for the river
        style: Dict of SVG style attributes (stroke, stroke-width, opacity, etc.)
    """
    if self.tree.size() == 0:
        return
    
    # Default style
    default_style = {
        'fill': 'none',
        'stroke': '#1565c0',
        'stroke-width': 3,
        'opacity': 0.7
    }
    
    if style:
        default_style.update(style)
    
    # Create StyleCSS from dict
    river_style = StyleCSS(
        layer,
        fill=default_style.get('fill', 'none'),
        stroke=default_style.get('stroke', '#1565c0'),
        stroke_width=default_style.get('stroke-width', 3),
        opacity=default_style.get('opacity', 0.7)
    )
    
    builder.add_style(river_style)
    
    # Generate SVG using existing svg() method
    svg_content = self.svg(styles=[river_style])
    builder.adjust(layer, svg_content)


## Factory

In [ ]:
#| export
from HexMagic.voronoi import generate_plate_terrain

In [ ]:
#| export
@patch
def recomputeClimate(self:Terrain):
    self.climate.configure(self)

In [ ]:
#| export
class Geology:

    def __init__(self,terrain,plates,rivers, num_plates=None, 
                          age = 0.,
                          
                          debug = False):
        self.terrain = terrain
        self.plates = plates
        self.rivers = rivers
        self.soil = SoilSystem.from_plates(terrain, plates, debug=debug)
        self.model = ErosionModel(self.terrain, self.soil, age=age)
        #self.model.erode(iterations=3, base_rate=3.0)
        

In [ ]:
#| export

class TerrainFactory:
    """Factory for creating terrains with realistic climate parameters."""
    
    PRESETS = {
    'tropical': ClimatePreset(
        name='Tropical',
        lat_range=(0, 15),
        base_temp_range=(26, 28),
        wind_speed=8.0,
        wind_dir=90.0,
        precip_base=0.2,
        nm=0.008,
        hw=2500.0,
        cw=0.004,
        conv_time=1000.0,
        fall_time=1000.0,
        lapse_rate=5.5,
        precip_bins=(1500, 2500, 4000, 6000),  # High rainfall expected
        description="Hot, humid, high rainfall. Trade winds from east."
    ),
    
    'subtropical': ClimatePreset(
        name='Subtropical',
        lat_range=(15, 30),
        base_temp_range=(20, 24),
        wind_speed=10.0,
        wind_dir=270.0,
        precip_base=0.12,
        nm=0.009,
        hw=2200.0,
        cw=0.003,
        conv_time=1200.0,
        fall_time=1200.0,
        lapse_rate=6.0,
        precip_bins=(600, 1000, 1800, 3000),  # Moderate
        description="Warm, moderate rainfall. Mediterranean-like."
    ),
    
    'mediterranean': ClimatePreset(
        name='Mediterranean',
        lat_range=(30, 45),
        base_temp_range=(14, 18),
        wind_speed=12.0,
        wind_dir=270.0,
        precip_base=0.08,
        nm=0.01,
        hw=2000.0,
        cw=0.0025,
        conv_time=1500.0,
        fall_time=1500.0,
        lapse_rate=6.5,
        precip_bins=(250, 500, 900, 1400),  # Drier
        description="Mild winters, dry summers. California-like."
    ),
    
    'temperate': ClimatePreset(
        name='Temperate',
        lat_range=(35, 55),
        base_temp_range=(8, 14),
        wind_speed=15.0,
        wind_dir=270.0,
        precip_base=0.15,
        nm=0.012,
        hw=1800.0,
        cw=0.003,
        conv_time=1500.0,
        fall_time=1500.0,
        lapse_rate=6.5,
        precip_bins=(800, 1400, 2200, 3500),  # Wet
        description="Four seasons, moderate rainfall. Pacific Northwest-like."
    ),
    
    'continental': ClimatePreset(
        name='Continental',
        lat_range=(40, 60),
        base_temp_range=(2, 10),
        wind_speed=12.0,
        wind_dir=270.0,
        precip_base=0.08,
        nm=0.013,
        hw=1600.0,
        cw=0.002,
        conv_time=2000.0,
        fall_time=2000.0,
        lapse_rate=7.0,
        precip_bins=(200, 400, 700, 1200),  # Dry interior
        description="Hot summers, cold winters, low rainfall. Central Asia-like."
    ),
    
    'subarctic': ClimatePreset(
        name='Subarctic',
        lat_range=(55, 70),
        base_temp_range=(-5, 5),
        wind_speed=10.0,
        wind_dir=270.0,
        precip_base=0.05,
        nm=0.015,
        hw=1400.0,
        cw=0.0015,
        conv_time=2500.0,
        fall_time=2500.0,
        lapse_rate=7.5,
        precip_bins=(150, 300, 500, 800),  # Low precipitation
        description="Long cold winters, short cool summers. Alaska-like."
    ),
    
    'arctic': ClimatePreset(
        name='Arctic',
        lat_range=(70, 85),
        base_temp_range=(-15, -5),
        wind_speed=8.0,
        wind_dir=0.0,
        precip_base=0.02,
        nm=0.018,
        hw=1200.0,
        cw=0.001,
        conv_time=3000.0,
        fall_time=3000.0,
        lapse_rate=8.0,
        precip_bins=(50, 100, 200, 400),  # Very dry
        description="Extremely cold, very low precipitation. Ice cap climate."
    ),
    
    'monsoon': ClimatePreset(
        name='Monsoon',
        lat_range=(10, 25),
        base_temp_range=(24, 28),
        wind_speed=12.0,
        wind_dir=135.0,
        precip_base=0.25,
        nm=0.007,
        hw=2800.0,
        cw=0.005,
        conv_time=800.0,
        fall_time=800.0,
        lapse_rate=5.5,
        precip_bins=(2000, 3500, 5500, 8000),  # Extreme rainfall
        description="Extreme seasonal rainfall. India/Southeast Asia-like."
    ),
    
    'oceanic': ClimatePreset(
        name='Oceanic',
        lat_range=(40, 60),
        base_temp_range=(8, 12),
        wind_speed=18.0,
        wind_dir=270.0,
        precip_base=0.2,
        nm=0.01,
        hw=2000.0,
        cw=0.004,
        conv_time=1000.0,
        fall_time=1000.0,
        lapse_rate=6.0,
        precip_bins=(1000, 1800, 2800, 4000),  # Very wet
        description="Mild, wet year-round. British Isles-like."
    ),
}
    
    
    @staticmethod
    def create_ocean_world(bounds: MapRect, 
                        preset: str = 'temperate',
                        radius: float = 15,
                        lon_span: float = 10.0,
                        custom_params: dict = None,
                        # Plate params
                        num_plates: int = None, 
                        subdivisions: int = 3,
                        ocean_fraction: float = 0.4,
                        oceanic_sides: list = ['N'],
                        edge_factor: float = 1.5,
                        # Terrain character
                        terrain_age: str = 'middle',  # 'young', 'middle', 'old'
                        formation_type: str = 'ocean_distance',
                        elevation_scale: float = 1.0,
                        # Erosion
                        erosion_age: float = 0.25,
                        num_lakes: int = 4,
                        seed: int = None,
                        debug: bool = False) -> Geology:
        """
        Create a world with tectonic plates and climate.
        
        Args:
            bounds: MapRect for the hex grid
            preset: Climate preset name (see PRESETS)
            radius: Hex radius
            lon_span: Longitude span in degrees
            custom_params: Override specific climate preset parameters
            num_plates: Number of tectonic plates (None for blank ocean)
            subdivisions: Plate subdivision depth for detail
            ocean_fraction: Fraction of plates marked oceanic
            oceanic_sides: List of sides ['N','E','S','W'] that are ocean
            edge_factor: How far from edge to mark as oceanic
            terrain_age: 'young' (sharp), 'middle', 'old' (eroded) - affects initial terrain
            formation_type: 'ocean_distance', 'ridge', 'volcanic', 'rolling'
            elevation_scale: Multiplier for elevations
            erosion_age: Age for erosion model (0-1, higher = more eroded)
            num_lakes: how much do we carve
            seed: Random seed
            debug: Print debug info
        
        Returns:
            Geology with terrain, plates, soil, and erosion model
        """
        
        if preset not in TerrainFactory.PRESETS:
            raise ValueError(f"Unknown preset: {preset}. Available: {list(TerrainFactory.PRESETS.keys())}")
        
        climate_preset = TerrainFactory.PRESETS[preset]
        
        # Calculate geographic bounds
        lat_min, lat_max = climate_preset.lat_range
        lon_min = -lon_span / 2
        lon_max = lon_span / 2
        
        # Create terrain
        if num_plates is None:
            terrain = Terrain(bounds, radius=radius)
            terrain.elevations = np.zeros(len(terrain.elevations))  # All ocean
            plates = []
        else:
            terrain, plates = generate_plate_terrain(
                bounds, 
                radius=radius,
                num_plates=num_plates,
                subdivisions=subdivisions,
                ocean_fraction=ocean_fraction,
                oceanic_sides=oceanic_sides,
                edge_factor=edge_factor,
                age=terrain_age,
                formation_type=formation_type,
                elevation_scale=elevation_scale,
                seed=seed
            )

        
        
        # Set geographic bounds
        terrain.geo = GeoBounds(
            lat_min=lat_min,
            lat_max=lat_max,
            lon_min=lon_min,
            lon_max=lon_max
        )
        
        # Compute hex coordinates
        terrain._compute_hex_coordinates()

        terrain.climate = climate_preset

        if num_lakes is not None:
            rivers = terrain.carve_to_ocean( num_lakes=num_lakes)
        else:
            rivers = []
            
        world = Geology(terrain, plates=plates, age=erosion_age, rivers = rivers, debug=debug)

        # Apply custom parameter overrides
        if custom_params:
            for key, value in custom_params.items():
                if hasattr(climate_preset, key):
                    setattr(climate_preset, key, value)
        
        if debug:
            print(f"\n=== TERRAIN FACTORY ===")
            print(f"Preset: {climate_preset.name}")
            print(f"Description: {climate_preset.description}")
            print(f"Latitude: {lat_min}° to {lat_max}°")
            print(f"Longitude: {lon_min}° to {lon_max}°")
            print(f"Grid: {terrain.hexGrid.nRows} x {terrain.hexGrid.nCols} hexes")
            print(f"Terrain age: {terrain_age}, Formation: {formation_type}")
            print(f"Base temperature range: {climate_preset.base_temp_range[0]}°C to {climate_preset.base_temp_range[1]}°C")
            print(f"Wind: {climate_preset.wind_speed} m/s from {climate_preset.wind_dir}°")
        
        return world

    
    
    @staticmethod
    def list_presets():
        """Print all available climate presets."""
        print("\n=== AVAILABLE CLIMATE PRESETS ===\n")
        for name, preset in TerrainFactory.PRESETS.items():
            print(f"{name.upper()}")
            print(f"  Latitude: {preset.lat_range[0]}° to {preset.lat_range[1]}°")
            print(f"  Temperature: {preset.base_temp_range[0]}°C to {preset.base_temp_range[1]}°C")
            print(f"  {preset.description}")
            print()


so I redid generate_plate_terrain. can you update create_ocean_world and take out parameters we don't need

``
def generate_plate_terrain(
    bounds, 
    radius=20,
    # Plate generation
    num_plates=10,
    subdivisions=3,
    min_subplate_size=3,
    # Ocean configuration  
    ocean_fraction=0.4,
    oceanic_sides=[],
    edge_factor=1.5,
    # Terrain character
    age='middle',  # 'young', 'middle', 'old'
    formation_type='ocean_distance',  # 'ridge', 'volcanic', 'rolling'
    # Fine-tuning
    elevation_scale=1.0,
    seed=None
):
    """
    Generate terrain based on tectonic plates.
    
    Args:
        bounds: MapRect defining terrain bounds
        radius: Hex radius
        num_plates: Initial number of large plates (for coastlines)
        subdivisions: Depth of plate subdivision for detail
        min_subplate_size: Minimum hex count per subplate
        ocean_fraction: Fraction of initial plates marked oceanic
        oceanic_sides: List of sides ['N','E','S','W'] that are ocean
        edge_factor: How far from edge to mark as oceanic
        age: 'young' (sharp), 'middle', 'old' (eroded)
        formation_type: 'ocean_distance', 'ridge', 'volcanic', 'rolling'
        elevation_scale: Multiplier for all elevations
        seed: Random seed
    
    Returns:
        (Terrain, plates) tuple
    """
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)
    
    # Age presets
    age_params = {
        'young':  {'slope': 30, 'variation': 50, 'smoothing': 1, 'peak_height': 200},
        'middle': {'slope': 20, 'variation': 40, 'smoothing': 3, 'peak_height': 150},
        'old':    {'slope': 10, 'variation': 20, 'smoothing': 6, 'peak_height': 80},
    }
    params = age_params.get(age, age_params['middle'])
    
    # Create terrain and grid
    terrain = Terrain(bounds, radius=radius)
    grid = terrain.hexGrid
    
    # Phase 1: Generate coarse plates for coastlines
    coarse_plates = Plate.voronoi(grid, num_plates)
    
    # Mark oceanic plates
    coarse_plates = mark_oceanic_plates(coarse_plates, ocean_fraction=ocean_fraction, seed=seed)
    if oceanic_sides:
        mark_oceanic_by_sides(coarse_plates, grid, factor=edge_factor, oceanic_sides=oceanic_sides)
    
    # Phase 2: Subdivide for detail (preserving plate kind)
    plates = []
    for plate in coarse_plates:
        sub = plate.subdivide(depth=subdivisions, min_size=min_subplate_size)
        plates.extend(sub)
    
    # Calculate ocean distances
    grid.calculate_distances_from_ocean(plates)
    
    # Get boundaries for ridge-based terrain
    boundaries = Plate.boundaries(plates)
    
    # Initialize elevations
    elevations = np.zeros(len(grid.hexes))
    
    # Generate elevations based on formation type
    if formation_type == 'ocean_distance':
        elevations = _elevation_ocean_distance(grid, plates, params, elevation_scale)
    elif formation_type == 'ridge':
        elevations = _elevation_ridge(grid, plates, boundaries, params, elevation_scale)
    elif formation_type == 'volcanic':
        elevations = _elevation_volcanic(grid, plates, params, elevation_scale)
    elif formation_type == 'rolling':
        elevations = _elevation_rolling(grid, plates, params, elevation_scale)
    else:
        raise ValueError(f"Unknown formation_type: {formation_type}")
    
    # Apply smoothing based on age
    for _ in range(params['smoothing']):
        elevations = _smooth_elevations(grid, elevations)
    
    terrain.elevations = elevations
    terrain.colorMap()
    
    return terrain, plates


def _elevation_ocean_distance(grid, plates, params, scale):
    """Original approach: elevation based on distance from ocean."""
    elevations = np.zeros(len(grid.hexes))
    
    for plate in plates:
        if plate.kind == PlateKind.oceanic:
            base = -100 * scale
            var = params['variation'] * 0.5
        else:
            base = (50 + plate.oceanDistance * params['slope']) * scale
            var = params['variation']
        
        for idx in plate.hexes:
            elevations[idx] = base + np.random.uniform(-var, var) * scale
    
    return elevations


def _elevation_ridge(grid, plates, boundaries, params, scale):
    """Mountains form at plate boundaries."""
    elevations = np.zeros(len(grid.hexes))
    
    # Build plate lookup
    plate_map = {}
    for plate in plates:
        for idx in plate.hexes:
            plate_map[idx] = plate
    
    for i, hx in enumerate(grid.hexes):
        plate = plate_map.get(i)
        if plate is None:
            continue
        
        pos = grid.index_to_hexposition(i)
        
        # Distance to nearest boundary
        if boundaries:
            dist_to_boundary = min(pos.distance(b) for b in boundaries)
        else:
            dist_to_boundary = 10
        
        if plate.kind == PlateKind.oceanic:
            # Ocean: slight trench near boundaries
            base = -80 * scale
            boundary_effect = -max(0, 2 - dist_to_boundary) * 10 * scale
        else:
            # Continental: ridges at boundaries
            base = 30 * scale
            ridge_height = max(0, 4 - dist_to_boundary) * params['peak_height'] / 4 * scale
            boundary_effect = ridge_height
        
        var = params['variation'] * 0.5
        elevations[i] = base + boundary_effect + np.random.uniform(-var, var) * scale
    
    return elevations


def _elevation_volcanic(grid, plates, params, scale):
    """Peaks at plate centroids (volcanic islands, mountain cores)."""
    elevations = np.zeros(len(grid.hexes))
    
    for plate in plates:
        centroid = plate.centroid_position()
        
        if plate.kind == PlateKind.oceanic:
            base = -60 * scale
            peak = params['peak_height'] * 0.3 * scale  # Underwater volcanoes
        else:
            base = 40 * scale
            peak = params['peak_height'] * scale
        
        for idx in plate.hexes:
            pos = grid.index_to_hexposition(idx)
            dist = pos.distance(centroid)
            
            # Peak at center, falls off
            height = max(0, peak - dist * params['slope'])
            var = params['variation'] * 0.3
            elevations[idx] = base + height + np.random.uniform(-var, var) * scale
    
    return elevations


def _elevation_rolling(grid, plates, params, scale):
    """Gentle rolling terrain with minimal features."""
    elevations = np.zeros(len(grid.hexes))
    
    for plate in plates:
        if plate.kind == PlateKind.oceanic:
            base = -30 * scale
        else:
            base = 20 + plate.oceanDistance * params['slope'] * 0.3
            base *= scale
        
        var = params['variation'] * 0.7
        for idx in plate.hexes:
            elevations[idx] = base + np.random.uniform(-var, var) * scale
    
    return elevations


def _smooth_elevations(grid, elevations):
    """Single smoothing pass averaging with neighbors."""
    smoothed = elevations.copy()
    for i in range(len(grid.hexes)):
        neighbors = grid.neighborsOf(i)
        if neighbors:
            neighbor_vals = [elevations[n] for n in neighbors if n < len(elevations)]
            if neighbor_vals:
                smoothed[i] = 0.7 * elevations[i] + 0.3 * np.mean(neighbor_vals)
    return smoothed


Terrain.fromSeeds = generate_plate_terrain
```

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Here's the updated `create_ocean_world`:

```python
@staticmethod
def create_ocean_world(bounds: MapRect, 
                      preset: str = 'temperate',
                      radius: float = 15,
                      lon_span: float = 10.0,
                      custom_params: dict = None,
                      # Plate params
                      num_plates: int = None, 
                      subdivisions: int = 3,
                      ocean_fraction: float = 0.4,
                      oceanic_sides: list = ['N'],
                      edge_factor: float = 1.5,
                      # Terrain character
                      terrain_age: str = 'middle',  # 'young', 'middle', 'old'
                      formation_type: str = 'ocean_distance',
                      elevation_scale: float = 1.0,
                      # Erosion
                      erosion_age: float = 0.25,
                      seed: int = None,
                      debug: bool = False) -> Geology:
    """
    Create a world with tectonic plates and climate.
    
    Args:
        bounds: MapRect for the hex grid
        preset: Climate preset name (see PRESETS)
        radius: Hex radius
        lon_span: Longitude span in degrees
        custom_params: Override specific climate preset parameters
        num_plates: Number of tectonic plates (None for blank ocean)
        subdivisions: Plate subdivision depth for detail
        ocean_fraction: Fraction of plates marked oceanic
        oceanic_sides: List of sides ['N','E','S','W'] that are ocean
        edge_factor: How far from edge to mark as oceanic
        terrain_age: 'young' (sharp), 'middle', 'old' (eroded) - affects initial terrain
        formation_type: 'ocean_distance', 'ridge', 'volcanic', 'rolling'
        elevation_scale: Multiplier for elevations
        erosion_age: Age for erosion model (0-1, higher = more eroded)
        seed: Random seed
        debug: Print debug info
    
    Returns:
        Geology with terrain, plates, soil, and erosion model
    """
    
    if preset not in TerrainFactory.PRESETS:
        raise ValueError(f"Unknown preset: {preset}. Available: {list(TerrainFactory.PRESETS.keys())}")
    
    climate_preset = TerrainFactory.PRESETS[preset]
    
    # Calculate geographic bounds
    lat_min, lat_max = climate_preset.lat_range
    lon_min = -lon_span / 2
    lon_max = lon_span / 2
    
    # Create terrain
    if num_plates is None:
        terrain = Terrain(bounds, radius=radius)
        terrain.elevations = np.zeros(len(terrain.elevations))  # All ocean
        plates = []
    else:
        terrain, plates = generate_plate_terrain(
            bounds, 
            radius=radius,
            num_plates=num_plates,
            subdivisions=subdivisions,
            ocean_fraction=ocean_fraction,
            oceanic_sides=oceanic_sides,
            edge_factor=edge_factor,
            age=terrain_age,
            formation_type=formation_type,
            elevation_scale=elevation_scale,
            seed=seed
        )
    
    # Set geographic bounds
    terrain.geo = GeoBounds(
        lat_min=lat_min,
        lat_max=lat_max,
        lon_min=lon_min,
        lon_max=lon_max
    )
    
    # Compute hex coordinates
    terrain._compute_hex_coordinates()

    terrain.climate = climate_preset
    world = Geology(terrain, plates=plates, age=erosion_age, debug=debug)

    # Apply custom parameter overrides
    if custom_params:
        for key, value in custom_params.items():
            if hasattr(climate_preset, key):
                setattr(climate_preset, key, value)
    
    if debug:
        print(f"\n=== TERRAIN FACTORY ===")
        print(f"Preset: {climate_preset.name}")
        print(f"Description: {climate_preset.description}")
        print(f"Latitude: {lat_min}° to {lat_max}°")
        print(f"Longitude: {lon_min}° to {lon_max}°")
        print(f"Grid: {terrain.hexGrid.nRows} x {terrain.hexGrid.nCols} hexes")
        print(f"Terrain age: {terrain_age}, Formation: {formation_type}")
        print(f"Base temperature range: {climate_preset.base_temp_range[0]}°C to {climate_preset.base_temp_range[1]}°C")
        print(f"Wind: {climate_preset.wind_speed} m/s from {climate_preset.wind_dir}°")
    
    return world
```

Key changes:
- Removed `ruggedness` and the manual `slope`/`variation` calculation
- Renamed `factor` → `edge_factor` 
- Renamed `age` → `erosion_age` (for the erosion model)
- Added `terrain_age` for the new plate terrain generation ('young'/'middle'/'old')
- Added `formation_type`, `subdivisions`, `elevation_scale`, `seed`
- Cleaner parameter grouping with comments

In [ ]:

def island_demo_fixed(debug=False):
    """Create a tropical island with three volcanoes and downsampled rivers."""
    
    # 1. Create blank ocean world with tropical preset
    bounds = MapRect(MapCord(0, 0), MapSize(800, 800))
    world = TerrainFactory.create_ocean_world(
        bounds=bounds,
        preset='tropical',
        radius=15,
        lon_span=5.0,
        num_plates=16,
        oceanic_sides=['N'],
        debug = debug
    )
    

    terrain = world.terrain
    if debug:
        print("\n=== COMPUTING CLIMATE ===")
    terrain.climate.configure(terrain,debug=debug)
    
    
    # 6. Visualize original
    if debug:
        print("\n=== RENDERING ORIGINAL ===")
    terrain.colorMap()
    terrain.hexGrid.update()
    terrain.add_climate_overlay()
    
    
    # 7. Downsample terrain (including flow)
    if debug:
        
        print("\n=== DOWNSAMPLING ===")
    smaller = terrain.downsample_climate(0.5)
    smaller.hexGrid.adjustRadius(20)
    
    # 9. Visualize downsampled version
    smaller.colorMap()
    smaller.hexGrid.update()
    smaller.add_climate_overlay()

    return smaller, world

isf, isfWorld = island_demo_fixed()
isf.colorMap
isf.hexGrid.update()
isf.hexGrid.builder.show()

Done at iter 1: 4 lakes


In [ ]:
isf = isfWorld.terrain
isf.hexGrid.builder.adjust("soil", isfWorld.soil.soilOverlay())
isf.hexGrid.builder.show()

In [ ]:
#| export
@patch
def bayAreaMap(self:TerraDemo,debug = False):
    """Load Maui terrain with proper geographic bounds."""
    with open("data/templates/BayArea.txt", "r") as f:
        terrain = Terrain.decode(f.read())
    
    # Set Maui's geographic bounds
    terrain.geo = GeoBounds(
        lat_min=37.57,   # 
        lat_max=38.03,   # 
        lon_min=-122.69, # 
        lon_max=-121.97  # 
    )
    
    # Compute hex coordinates
    terrain._compute_hex_coordinates()
    
    terrain.climate =  ClimatePreset(
            name='Mediterranean',
            lat_range=(30, 45),
            base_temp_range=(14, 18),
            wind_speed=12.0,
            wind_dir=270.0,  # Westerlies
            precip_base=0.08,
            nm=0.01,
            hw=2000.0,
            cw=0.0025,
            conv_time=1500.0,
            fall_time=1500.0,
            lapse_rate=6.5,
            description="Mild winters, dry summers. California-like."
        )

    return terrain

In [ ]:
bsm = TerraDemo().bayAreaMap()
bsm.recomputeClimate()
bsm.colorMap()
bsm.hexGrid.update()
bsm.hexGrid.builder.show()